In [ ]:
# Convert this to a grofile with a truncated octahedral box
#     box type: octahedron
#     make box boundaries 1.0 nm from protein
#     (the protein will be centered in the box automatically)

###editconf -f peptoid.gro -o peptoid_boxed.gro -bt cubic -d 0.8

# solvate the strutcure
#     options: 

###genbox -cp peptoid_boxed.gro -cs spc216.gro -o peptoid_solvated.gro

# build a topology file by hand
###exit

# make an index file
make_ndx -f peptoid_solvated.gro -o index.ndx

# Add counterions at ~100 mM to neutralize the system
### We need a tpr so that genion will work!
grompp -f mdp/minimize.mdp -c peptoid_solvated.gro -p peptoid_solvated.top -n index.ndx -o minimize.tpr
### NOTE that genion will overwite your topology file (!) recalcultig the numbers of SOL, Na and Cl molecules
genion -s minimize.tpr -n index.ndx -p peptoid_solvated.top -o solvent_ions.gro -pname NA -nname CL  -neutral -conc 0.1
make_ndx -f solvent_ions.gro -o index.ndx

# Minimize
grompp -f mdp/minimize.mdp -c solvent_ions.gro -p peptoid_solvated.top -n index.ndx -o minimize.tpr
mdrun -v -nt 4 -s minimize.tpr -c minimized.gro

# Equilibrate with pressure coupling on (Berendsen)
grompp -f mdp/equil.mdp -c minimized.gro -p peptoid_solvated.top -n index.ndx -o equil.tpr
mdrun -v -nt 4 -s equil.tpr -c peptoid_solvated_equilibrated.gro

# Visulize the gro file

In [2]:
import py3Dmol

# Load the GRO TPR file content
with open('19AE1-4-A_protonated_GMX.gro', 'r') as file:  # Removed the extra space in the filename
    gro_content = file.read()

# Initialize the viewer
view = py3Dmol.view()

# Add the GRO content to the viewer
view.addModel(gro_content, 'gro')  # Ensure 'gro' matches the format of your file

# Set the style for better visualization
view.setStyle({'stick': {}})

# Zoom to fit the molecule in the viewer
view.zoomTo()

# Show the viewer
view.show()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# Prepare the tpr file using Gromacs

In [4]:
!gmx -h

                   :-) GROMACS - gmx, 2023.3-conda_forge (-:

Executable:   /Users/starwingchen/anaconda3/envs/gromacs-env/bin.AVX2_256/gmx
Data prefix:  /Users/starwingchen/anaconda3/envs/gromacs-env
Working dir:  /Users/starwingchen/Voelz_Lab/git/peptoid_24summer/test-output/19AE1-4-A/protonated/19AE1-4-A_protonated.amb2gmx
Command line:
  gmx -h

SYNOPSIS

gmx [-[no]h] [-[no]quiet] [-[no]version] [-[no]copyright] [-nice <int>]
    [-[no]backup]

OPTIONS

Other options:

 -[no]h                     (no)
           Print help and quit
 -[no]quiet                 (no)
           Do not print common startup info or quotes
 -[no]version               (no)
           Print extended version information and quit
 -[no]copyright             (no)
           Print copyright information on startup
 -nice   <int>              (19)
           Set the nicelevel (default depends on command)
 -[no]backup                (yes)
           Write backups if output files exist

Additional help is availabl

In [8]:
# make a cubic 1A box

!gmx editconf -f 19AE1-4-A_protonated_GMX.gro -o 19AE1-4-A_protonated_GMX_boxed.gro -bt cubic -d 1.0

               :-) GROMACS - gmx editconf, 2023.3-conda_forge (-:

Executable:   /Users/starwingchen/anaconda3/envs/gromacs-env/bin.AVX2_256/gmx
Data prefix:  /Users/starwingchen/anaconda3/envs/gromacs-env
Working dir:  /Users/starwingchen/Voelz_Lab/git/peptoid_24summer/test-output/19AE1-4-A/protonated/19AE1-4-A_protonated.amb2gmx
Command line:
  gmx editconf -f 19AE1-4-A_protonated_GMX.gro -o 19AE1-4-A_protonated_GMX_boxed.gro -bt cubic -d 1.0

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 83 atoms
Volume: 22734.8 nm^3, corresponds to roughly 10230600 electrons
No velocities found
    system size :  1.663  1.454  1.175 (nm)
    diameter    :  1.786               (nm)
    center      :  0.369 -0.027 -0.193 (nm)
    box vectors : 33.262 29.070 23.512 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :22734.79               (nm^3)
    shift       :  1.524  1.920  2.087 (nm)
new center      :  1.893  1.893  1.893 (nm

In [11]:
# Fill the box with water as solvent 
!gmx solvate -cp 19AE1-4-A_protonated_GMX_boxed.gro -cs spc216.gro -o 19AE1-4-A_protonated_GMX_solv.gro

               :-) GROMACS - gmx solvate, 2023.3-conda_forge (-:

Executable:   /Users/starwingchen/anaconda3/envs/gromacs-env/bin.AVX2_256/gmx
Data prefix:  /Users/starwingchen/anaconda3/envs/gromacs-env
Working dir:  /Users/starwingchen/Voelz_Lab/git/peptoid_24summer/test-output/19AE1-4-A/protonated/19AE1-4-A_protonated.amb2gmx
Command line:
  gmx solvate -cp 19AE1-4-A_protonated_GMX_boxed.gro -cs spc216.gro -o 19AE1-4-A_protonated_GMX_solv.gro

Reading solute configuration
Reading solvent configuration

Initialising inter-atomic distances...

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: F

In [12]:
# make an index file
!gmx make_ndx -f 19AE1-4-A_protonated_GMX_solv.gro -o index.ndx

               :-) GROMACS - gmx make_ndx, 2023.3-conda_forge (-:

Executable:   /Users/starwingchen/anaconda3/envs/gromacs-env/bin.AVX2_256/gmx
Data prefix:  /Users/starwingchen/anaconda3/envs/gromacs-env
Working dir:  /Users/starwingchen/Voelz_Lab/git/peptoid_24summer/test-output/19AE1-4-A/protonated/19AE1-4-A_protonated.amb2gmx
Command line:
  gmx make_ndx -f 19AE1-4-A_protonated_GMX_solv.gro -o index.ndx


Reading structure file
Going to read 0 old index file(s)
Analysing residue names:
There are:     4      Other residues
There are:     1    Protein residues
There are:  1672      Water residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing Protein...

  0 System              :  5099 atoms
  1 Other               :    80 atoms
  2 NAE                 :    34 atoms
  3 NPE                 :    46 atoms
  4 Protein             :     3 atoms
  5 Protein-H           :     1 atoms
  6 C-alpha             :     0 atoms
  7 Backbone     

In [5]:
# Add counterions at ~100 mM to neutralize the system
### We need a tpr so that genion will work!
!gmx grompp -f minimize.mdp -c 19AE1-4-A_protonated_GMX_solv.gro -p 19AE1-4-A_protonated_GMX_solv.top -n index.ndx -o minimize.tpr

                :-) GROMACS - gmx grompp, 2023.3-conda_forge (-:

Executable:   /Users/starwingchen/anaconda3/envs/gromacs-env/bin.AVX2_256/gmx
Data prefix:  /Users/starwingchen/anaconda3/envs/gromacs-env
Working dir:  /Users/starwingchen/Voelz_Lab/git/peptoid_24summer/test-output/19AE1-4-A/protonated/19AE1-4-A_protonated.amb2gmx
Command line:
  gmx grompp -f minimize.mdp -c 19AE1-4-A_protonated_GMX_solv.gro -p 19AE1-4-A_protonated_GMX_solv.top -n index.ndx -o minimize.tpr


-------------------------------------------------------
Program:     gmx grompp, version 2023.3-conda_forge
Source file: src/gromacs/commandline/cmdlineparser.cpp (line 271)
Function:    void gmx::CommandLineParser::parse(int *, char **)

Error in user input:
Invalid command-line options
  In command-line option -p
    File '19AE1-4-A_protonated_GMX_solv.top' does not exist or is not
    accessible.
    The file could not be opened.
      Reason: No such file or directory
      (call to fopen() returned error code 